In [ ]:
!pip install backtesting 
!pip install ta-lib-binary
!pip install yfinance

In [1]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import talib as ta
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import datetime
import numpy as np

/usr/local/lib/python3.8/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [62]:
# ticker and the start and end dates for testing
ticker =  input("Enter ticker: ")
START = datetime.datetime(2013, 1, 1)
END = datetime.datetime(2023, 1, 1)
YEARS = (END - START).days / 365.25

Enter ticker: ^GDAXI


In [63]:
data=yf.download( ticker , start=START,end=END)
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-01-02,7689.459961,7789.939941,7689.459961,7778.779785,7778.779785,140345500
2013-01-03,7762.129883,7777.020020,7737.979980,7756.439941,7756.439941,86174100
2013-01-04,7740.990234,7779.669922,7727.549805,7776.370117,7776.370117,101577000
2013-01-07,7760.009766,7767.549805,7713.419922,7732.660156,7732.660156,151695400
2013-01-08,7710.700195,7754.379883,7683.479980,7695.830078,7695.830078,130425500
...,...,...,...,...,...,...
2022-12-23,13945.589844,14000.679688,13874.500000,13940.929688,13940.929688,28738700
2022-12-27,14047.419922,14063.139648,13966.349609,13995.099609,13995.099609,22975000
2022-12-28,14013.719727,14018.469727,13914.620117,13925.599609,13925.599609,27583800


In [64]:
def MACD(close, n1, n2, ns):
    #n1-n2
    macd, macdsignal, macdhist = ta.MACD(close, fastperiod=n1, slowperiod=n2, signalperiod=ns)
    return macd, macdsignal

class MACDCross(Strategy):
    n1 = 12 
    n2 = 26 
    ns = 9 

    def init(self):
        self.macd, self.macdsignal = self.I(MACD, self.data.Close, self.n1, self.n2, self.ns)

    def next(self): 
        if crossover(self.macd, self.macdsignal): 
            self.buy()
        elif crossover(self.macdsignal, self.macd):            
          self.position.close()

In [65]:

bt = Backtest(
    data,
    MACDCross,
    cash=100_000,
    commission=0.01,
    margin=1.0,
    trade_on_close=True,
    exclusive_orders=True)

output = bt.run()

In [66]:
output2=bt.optimize(n1=range(10, 100, 10),n2=range(10, 300, 10),ns=range(10, 50, 5), maximize='Return [%]', method='grid')
print(output2)
bt.plot()

/usr/local/lib/python3.8/dist-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 2088 configurations.
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/7 [00:00<?, ?it/s]

Start                     2013-01-02 00:00:00
End                       2022-12-30 00:00:00
Duration                   3649 days 00:00:00
Exposure Time [%]                   45.418641
Equity Final [$]                128137.727354
Equity Peak [$]                 137611.894502
Return [%]                          28.137727
Buy & Hold Return [%]               78.994524
Return (Ann.) [%]                    2.498302
Volatility (Ann.) [%]               11.110175
Sharpe Ratio                         0.224866
Sortino Ratio                        0.320349
Calmar Ratio                          0.13372
Max. Drawdown [%]                  -18.683057
Avg. Drawdown [%]                   -2.935185
Max. Drawdown Duration     1185 days 00:00:00
Avg. Drawdown Duration      103 days 00:00:00
# Trades                                   15
Win Rate [%]                             40.0
Best Trade [%]                      21.265346
Worst Trade [%]                     -9.355502
Avg. Trade [%]                    

Row(id='8795', ...)

In [67]:
output2._strategy

<Strategy MACDCross(n1=50,n2=270,ns=45)>

In [68]:
#These are the trades that would executed with this strategy.
output2['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,9,356,382,10049.621318,9772.669922,-2492.562568,-0.027558,2014-06-02,2014-07-08,36 days
1,9,483,597,10074.618818,11594.280273,13676.953096,0.150841,2014-11-27,2015-05-18,172 days
2,10,721,759,10923.604697,10214.019531,-7095.851660,-0.064959,2015-11-09,2016-01-06,58 days
3,10,818,881,10065.164863,9612.269531,-4528.953320,-0.044996,2016-03-31,2016-06-29,90 days
4,9,894,1137,10163.761182,12325.120117,19452.230420,0.212653,2016-07-18,2017-06-30,347 days
5,9,1206,1260,13085.499834,13072.790039,-114.388154,-0.000971,2017-10-06,2017-12-22,77 days
6,8,1279,1284,13695.195605,13197.709961,-3979.885156,-0.036326,2018-01-23,2018-01-30,7 days
7,8,1353,1385,13153.098818,12234.339844,-7350.071797,-0.069851,2018-05-10,2018-06-26,47 days
8,9,1540,1667,11437.966924,11845.410156,3666.989092,0.035622,2019-02-06,2019-08-08,183 days
9,8,1695,1806,12496.336455,12367.459961,-1031.011953,-0.010313,2019-09-17,2020-02-27,163 days
